In [1]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix

from keras.callbacks import EarlyStopping
from keras.models import Model, Sequential, load_model
from keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

Definição da SEED:

In [2]:
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED) # Numpy

# Carregando os dados

Os valores são números muito pequenos com muitas casas decimais, por isso é bom que o dataframe consiga representar isso também.

In [ ]:
pd.set_option('display.float_format', '{:.20f}'.format)

Carregando os dados:

In [ ]:
df_benign = pd.read_csv("data/dados benignos/mensagens_benignas.csv")
df_benign

In [ ]:
df_malicious_random_dos = pd.read_csv("data/ataques/mensagens_maliciosas_random_dos.csv")
df_malicious_spoofing_zero_payload = pd.read_csv("data/ataques/mensagens_maliciosas_spoofing_zero_payload.csv")
df_malicious_zero_dos = pd.read_csv("data/ataques/mensagens_maliciosas_zero_dos.csv")

In [ ]:
df_malicious_random_dos

In [ ]:
df_malicious_spoofing_zero_payload

In [ ]:
df_malicious_zero_dos

Manipualação da quantidade/proporção dos dados:

In [ ]:
# OPCIONAL!!!
len_min = min(len(df_benign),len(df_malicious_random_dos),len(df_malicious_spoofing_zero_payload),len(df_malicious_zero_dos))
NUM_OF_ATTACKS = 3

df_benign = df_benign.head(len_min * NUM_OF_ATTACKS)
df_malicious_random_dos = df_malicious_random_dos.head(len_min)
df_malicious_spoofing_zero_payload = df_malicious_spoofing_zero_payload.head(len_min)
df_malicious_zero_dos = df_malicious_zero_dos.head(len_min)

In [ ]:
del len_min

# Tratando dados

## Normalização dos dados

Criação e uso do scaler:

In [ ]:
scaler = MinMaxScaler()

scaler.fit(df_benign)

df_benign_scaled = pd.DataFrame(scaler.transform(df_benign), columns=df_benign.columns, index=df_benign.index)
df_malicious_random_dos_scaled = pd.DataFrame(scaler.transform(df_malicious_random_dos), columns=df_malicious_random_dos.columns, index=df_malicious_random_dos.index)
df_malicious_spoofing_zero_payload_scaled = pd.DataFrame(scaler.transform(df_malicious_spoofing_zero_payload), columns=df_malicious_spoofing_zero_payload.columns, index=df_malicious_spoofing_zero_payload.index)
df_malicious_zero_dos_scaled = pd.DataFrame(scaler.transform(df_malicious_zero_dos), columns=df_malicious_zero_dos.columns, index=df_malicious_zero_dos.index)

In [ ]:
del df_benign
del df_malicious_random_dos
del df_malicious_spoofing_zero_payload
del df_malicious_zero_dos

In [ ]:
df_benign_scaled

## Criando Labels

In [ ]:
list_labels_benign = [1] * len(df_benign_scaled)
list_labels_random_dos = [-1] * len(df_malicious_random_dos_scaled)
list_labels_spoofing_zero_payload = [-2] * len(df_malicious_spoofing_zero_payload_scaled)
list_labels_zero_dos = [-3] * len(df_malicious_zero_dos_scaled)

classifier_type = "mc"

## Criação de Janelas Temporais

Criação da função de divisão dos dados em janelas:

In [ ]:
def create_slicing_windows(data, labels, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step):
        a = data[i:(i + time_step)]
        X.append(a)
        Y.append(labels[i + time_step])
    return np.array(X), np.array(Y)

Definição do tamanho da janela:

In [ ]:
WINDOW_SIZE = 150

Criação de janelas deslizantes:

In [ ]:
benign_windows, benign_labels = create_slicing_windows(df_benign_scaled, list_labels_benign, WINDOW_SIZE)
del df_benign_scaled, list_labels_benign

malicious_random_dos_windows, malicious_random_dos_labels = create_slicing_windows(df_malicious_random_dos_scaled, list_labels_random_dos, WINDOW_SIZE)
del df_malicious_random_dos_scaled, list_labels_random_dos

malicious_spoofing_zero_payload_windows, malicious_spoofing_zero_payload_labels = create_slicing_windows(df_malicious_spoofing_zero_payload_scaled, list_labels_spoofing_zero_payload, WINDOW_SIZE)
del df_malicious_spoofing_zero_payload_scaled, list_labels_spoofing_zero_payload

malicious_zero_dos_windows, malicious_zero_dos_labels = create_slicing_windows(df_malicious_zero_dos_scaled, list_labels_zero_dos, WINDOW_SIZE)
del df_malicious_zero_dos_scaled, list_labels_zero_dos

In [ ]:
len(benign_windows), len(benign_windows[0])

In [ ]:
benign_windows

## Dividindo dados em Treino, Validação e Teste

Quantidade de dados benignos dividido pelo total:

In [ ]:
len(benign_windows) / (len(benign_windows) + len(malicious_random_dos_windows) + len(malicious_spoofing_zero_payload_windows) + len(malicious_zero_dos_windows))

Concatenação das janelas, em ordem:

In [ ]:
data = np.vstack((benign_windows, malicious_random_dos_windows, malicious_spoofing_zero_payload_windows, malicious_zero_dos_windows))

In [ ]:
data_malicious = np.vstack((malicious_random_dos_windows, malicious_spoofing_zero_payload_windows, malicious_zero_dos_windows))

In [ ]:
data_labels = np.hstack((benign_labels, malicious_random_dos_labels, malicious_spoofing_zero_payload_labels, malicious_zero_dos_labels))

In [ ]:
#del benign_windows
del malicious_random_dos_windows
del malicious_spoofing_zero_payload_windows
del malicious_zero_dos_windows

del benign_labels
del malicious_random_dos_labels
del malicious_spoofing_zero_payload_labels
del malicious_zero_dos_labels

Divisão em treino, validação e teste.

In [ ]:
train_data, val_test_data, train_labels, val_test_labels = train_test_split(data, data_labels, test_size=0.25, random_state=RANDOM_SEED)

In [ ]:
val_data, test_data, val_labels, test_labels = train_test_split(val_test_data, val_test_labels, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
del val_test_data, val_test_labels

Shape dos dados de treino:

In [ ]:
train_data.shape

# IAs

## LSTM (supervisionado)

Variável que define se é supervisionado ou não-supervisionado:

In [ ]:
s_OR_ns = "s"

Quantidade de features, para o modelo lidar com as entradas:

In [ ]:
FEATURES_COUNT = train_data.shape[2] # Número de features dos dados

Construção do modelo LSTM:

In [ ]:
# Construindo o modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(WINDOW_SIZE, FEATURES_COUNT)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1, kernel_regularizer=l2(0.0000001)))

# Compilando o modelo
model.compile(optimizer=Adam(learning_rate=0.004), loss='mse')
#model.compile(optimizer="adam", loss='mse')

Definição da paciência:

In [ ]:
PATIENCE = 4

Configuração do early stop:

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience= PATIENCE, restore_best_weights=True)

Definição da quantidade de épocas e da quantidade de batchs:

In [ ]:
EPOCHS = 50
BATCH_SIZE = 32 #padrão: batch_size=32

**TREINAMENTO DO MODELO:**

In [ ]:
# Treinando o modelo
history = model.fit(train_data, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_data, val_labels), callbacks=[early_stopping])

In [ ]:
# Avaliando o modelo no conjunto de teste
loss = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)

In [ ]:
model = load_model("model3_LSTM_s_mc_ws150_e50_p4_bs32.h5", compile=True)

In [ ]:
predicts = model.predict(test_data)

In [ ]:
predicts

In [ ]:
len(predicts[predicts > 0]) / (len(predicts[predicts < 0]) + len(predicts[predicts > 0]))

In [ ]:
len(test_labels[test_labels < 0]), len(test_labels[test_labels > 0])

In [ ]:
len(predicts[predicts < 0]), len(predicts[predicts > 0])

In [ ]:
len(predicts[predicts < 0]) / len(test_labels[test_labels < 0]), len(predicts[predicts > 0]) / len(test_labels[test_labels > 0])

In [ ]:
model.save(f'models/model_LSTM_{s_OR_ns}_{classifier_type}_ws{WINDOW_SIZE}_e{EPOCHS}_p{PATIENCE}_bs{BATCH_SIZE}.keras')

In [ ]:
dump(scaler, f"models/scales/scaler_model_LSTM_{s_OR_ns}_{classifier_type}_ws{WINDOW_SIZE}_e{EPOCHS}_p{PATIENCE}_bs{BATCH_SIZE}")

In [ ]:
#del scaler
del history
del predicts
del loss

## LSTM (não-supervisionado) (NÃO TERMINADO!!!)

In [ ]:
s_OR_ns = "ns"
classifier_type = "bc"

In [ ]:
FEATURES_COUNT = benign_windows.shape[2]

In [ ]:
# Construindo o modelo Autoencoder LSTM
inputs = Input(shape=(WINDOW_SIZE, FEATURES_COUNT))
encoded = LSTM(50, activation='tanh', return_sequences=False)(inputs)  # Codificador
decoded = RepeatVector(WINDOW_SIZE)(encoded)  # Decodificador
decoded = LSTM(FEATURES_COUNT, return_sequences=True)(decoded)

# Adicionando Dropout e Normalização em Batch
decoded = Dropout(0.1)(decoded)
decoded = BatchNormalization()(decoded)

# Camada de saída
outputs = Dense(FEATURES_COUNT, kernel_regularizer=l2(0.0000001))(decoded)

# Compilando o modelo
model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.004), loss='mse')

In [ ]:
PATIENCE = 4

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 32 #padrão: batch_size=32

In [ ]:
history = model.fit(benign_windows, benign_windows, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
predicts = model.predict(data_malicious)

In [ ]:
predicts

In [ ]:
def plot_comparison(original_data, predicted_data, sample_index=0):
    """
    Função para comparar visualmente os dados originais com as predições do modelo autoencoder.

    Parâmetros:
        original_data (numpy.array): Os dados originais.
        predicted_data (numpy.array): As previsões geradas pelo modelo autoencoder.
        sample_index (int): O índice da amostra a ser plotada.
    """
    plt.plot(original_data[sample_index], label='Original')
    plt.plot(predicted_data[sample_index], label='Predicted')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.title('Comparison of Original and Predicted Data')
    plt.legend()
    plt.show()

In [ ]:
plot_comparison(data_malicious, predicts)

In [ ]:
def classify_samples(original_data, predicted_data, threshold):
    """
    Função para classificar as amostras como benignas (1) ou malignas (-1) com base no erro de reconstrução.

    Parâmetros:
        original_data (numpy.array): Os dados originais.
        predicted_data (numpy.array): As previsões geradas pelo modelo autoencoder.
        threshold (float): O limiar de decisão para classificar as amostras.

    Retorna:
        numpy.array: Um array de classificação binária para cada amostra.
    """
    # Calcula o erro de reconstrução (MSE) para cada amostra
    reconstruction_errors = np.mean(np.square(original_data - predicted_data), axis=(1, 2))

    # Classifica as amostras com base no limiar
    classifications = np.where(reconstruction_errors < threshold, 1, -1)

    return classifications

In [ ]:
TRESHOLD = 0.01

In [ ]:
classifications = classify_samples(data_malicious, predicts, TRESHOLD)

In [ ]:
len(classifications[classifications == -1]) / len(classifications)

In [ ]:
model.save(f'models/model_LSTM_{s_OR_ns}_{classifier_type}_ws{WINDOW_SIZE}_e{EPOCHS}_p{PATIENCE}_bs{BATCH_SIZE}.keras')

In [ ]:
dump(scaler, f"models/scales/scaler_model_LSTM_{s_OR_ns}_{classifier_type}_ws{WINDOW_SIZE}_e{EPOCHS}_p{PATIENCE}_bs{BATCH_SIZE}")